In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load csv 
df_train = pd.read_csv('train.csv')
df_test_raw = pd.read_csv('test.csv')

# Create new column (Survived) in raw test data with a value of 0 on each row
df_test_raw['Survived'] = 0

In [ ]:
# Show the given row and columns in the datasets
shape_train = df_train.shape
shape_test = df_test_raw.shape
print('Train dataset shape: ')
print(shape_train)
print('Test dataset shape: ')
print(shape_test)

In [ ]:
# Show all info of the datasets
df_train.info()
df_test_raw.info()


In [ ]:
# Show total number of male and female onboard of the ship
df_train['Sex'].value_counts()

In [ ]:
# Show null values per column
df_train.isnull().sum()

In [ ]:
# Create new dataframe that only contains de PassengerId and Survived columns
# data = df_test_raw[['PassengerId', 'Survived']]

# Display the new dataframe
# display(data)

# Create a csv file of the dataframe called result
# data.to_csv('result.csv', index=False)

Hieronder code die mogelijk later gebruikt kan worden...

In [ ]:
# newdf = df_train[(df_train['Sex'] == 'male') & (df_train['Pclass'] == 3)]
# male_3 = newdf['Survived'].value_counts()
# tot_male = male_3[0] + male_3[1]
# male_3_lived = newdf[newdf['Survived'] == 1]
# print(male_3_lived)